# SQL Common Table Expressions

SQl is a very powerful language that allows the construction of very complex queries. But SQL only provides very limited support for structuting query in order to increase readability. So called "Common Table Expressions" (CTEs) are a common way to split up large SQL queries into smaller and more manageable chunks. This approach works with all SQL databases, not only with Spark SQL.

**Attention!** In Spark, CTEs will be optimized away, such that there is no difference in execution speed. Different (relational) databases might handle CTEs differently and they might represent an optimization barrier. Please consult the manual of your database before blindlingly using CTEs!

For this notebook, we will pick up the weather example and use Spark SQL to calculate some aggregates per year and country. We will not provide the most efficient implementation in order to make the whole structure more complicated to show the power of common table expressions.

In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

if not 'spark' in locals():
    spark = SparkSession.builder \
        .master("local[*]") \
        .config("spark.driver.memory","4G") \
        .getOrCreate()

spark

In [3]:
storageLocation = "s3://dimajix-training/data/weather"
#storageLocation = "/dimajix/data/weather-noaa-sample"

# 1. Register Temp Views

In a first step, we will load the input data via Spark. In order to use Spark SQL, we will also immediately register the loaded DataFrames as temp views.

## 1.1 Load Raw Data

Like before, we will load the raw measurement data and register it as a temp view called `raw_weather`.

In [8]:
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", f.lit(i)) for i in range(2003,2020)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

# Register Spark DataFrame as named temporary view called 'raw_weather'
raw_weather.createOrReplaceTempView("raw_weather")

In [9]:
# Display first 10 records
spark.sql("SELECT * FROM raw_weather LIMIT 10").toPandas()

,value,year
0,0494703160256242003010100003+55200-162717SY-MT...,2003
1,0228703160256242003010100174+55200-162730FM-16...,2003
2,044070316025624200301010053C+55200-162717FM-15...,2003
3,0071703160256242003010101009+55200-162717NSRDB...,2003
4,042770316025624200301010153C+55200-162717FM-15...,2003
5,0071703160256242003010102009+55200-162717NSRDB...,2003
6,046870316025624200301010253C+55200-162717FM-15...,2003
7,0071703160256242003010103009+55200-162717NSRDB...,2003
8,041570316025624200301010353C+55200-162717FM-15...,2003
9,0054703160256242003010104009+55200-162717NSRDB...,2003


## 2.2 Load Master Data

Now we will load the stations master data and register a temp view called `stations`.

In [14]:
stations = spark.read.csv(storageLocation + "/isd-history", header=True)

# Display first 10 records    
stations.createOrReplaceTempView("stations")

In [15]:
# Display first 10 records
spark.sql("SELECT * FROM stations LIMIT 10").toPandas()

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007018,99999,WXPOD 7018,None,None,None,+00.000,+000.000,+7018.0,20110309,20130730
1,007026,99999,WXPOD 7026,AF,None,None,+00.000,+000.000,+7026.0,20120713,20170822
2,007070,99999,WXPOD 7070,AF,None,None,+00.000,+000.000,+7070.0,20140923,20150926
3,008260,99999,WXPOD8270,None,None,None,+00.000,+000.000,+0000.0,20050101,20100920
4,008268,99999,WXPOD8278,AF,None,None,+32.950,+065.567,+1156.7,20100519,20120323
5,008307,99999,WXPOD 8318,AF,None,None,+00.000,+000.000,+8318.0,20100421,20100421
6,008411,99999,XM20,None,None,None,None,None,None,20160217,20160217
7,008414,99999,XM18,None,None,None,None,None,None,20160216,20160217
8,008415,99999,XM21,None,None,None,None,None,None,20160217,20160217
9,008418,99999,XM24,None,None,None,None,None,None,20160217,20160217


# 2. Using Intermediate Tables

Let us first perform the weather analysis step by step using more intermediate temporary tables. Of course, this approach is very specific to Apache Spark and not directly available in other databases. But additional tools like [dbt](https://getdbt.com) will provide similar capabilities for generic SQL databases.

## 2.1 Extract Measurements

In the first step, we need to extract all the interesting attributes from the raw measurement data. We do this by using `substring` and approproiate casts and/or scaling.

In [16]:
query = """
    SELECT
        year,
        substring(value,5,6) AS usaf,
        substring(value,11,5) AS wban,
        substring(value,16,8) AS `date`,
        substring(value,24,4) AS `time`,
        substring(value,42,5) AS report_type,
        substring(value,61,3) AS wind_direction,
        substring(value,64,1) AS wind_direction_qual,
        substring(value,65,1) AS wind_observation,
        CAST(substring(value,66,4) AS FLOAT) / 10.0 AS wind_speed,
        substring(value,70,1) AS wind_speed_qual,
        CAST(substring(value,88,5) AS FLOAT) / 10.0 AS air_temperature,
        substring(value,93,1) AS air_temperature_qual
    FROM raw_weather
"""

# Create a Spark DataFrame for the SQL query above
weather = spark.sql(query)
# Register the DataFrame as a temp view with name 'weather'
weather.createOrReplaceTempView("weather")

# Display the first 10 records from the newly created temp view 'weather'
spark.sql("SELECT * FROM weather LIMIT 10").toPandas()

,year,usaf,wban,date,time,report_type,wind_direction,wind_direction_qual,wind_observation,wind_speed,wind_speed_qual,air_temperature,air_temperature_qual
0,2003,703160,25624,20030101,0000,SY-MT,010,5,N,5.2,5,-0.6,5
1,2003,703160,25624,20030101,0017,FM-16,020,1,N,4.6,1,-2.0,1
2,2003,703160,25624,20030101,0053,FM-15,010,5,N,5.2,5,-2.8,5
3,2003,703160,25624,20030101,0100,NSRDB,999,9,9,999.9,9,999.9,9
4,2003,703160,25624,20030101,0153,FM-15,010,5,N,6.2,5,-2.2,5
5,2003,703160,25624,20030101,0200,NSRDB,999,9,9,999.9,9,999.9,9
6,2003,703160,25624,20030101,0253,FM-15,010,5,N,7.2,5,-3.3,5
7,2003,703160,25624,20030101,0300,NSRDB,999,9,9,999.9,9,999.9,9
8,2003,703160,25624,20030101,0353,FM-15,020,5,N,6.2,5,-1.1,5
9,2003,703160,25624,20030101,0400,NSRDB,999,9,9,999.9,9,999.9,9


## 2.2 Join Data

In the next step, we join the extracted measurement data with the stations master data. We use the two columns `usaf` and `wban` for joining. Since the join represents an enrichment of the measurements, we chose a *left join*. In order to access the extracted measurements, we can simply use the `weather` temp view we just created above.

In [17]:
query = """
    SELECT
        weather.*,
        CTRY AS country
    FROM weather
    LEFT JOIN stations ON weather.usaf = stations.usaf AND weather.wban = stations.wban
"""

# Create a Spark DataFrame for the SQL query above
joined_weather = spark.sql(query)
# Register the DataFrame as a temp view with name 'joined_weather'
joined_weather.createOrReplaceTempView("joined_weather")

# Display first 10 records from the newly created temp view 'joined_weather'
spark.sql("SELECT * FROM joined_weather LIMIT 10").toPandas()

,year,usaf,wban,date,time,report_type,wind_direction,wind_direction_qual,wind_observation,wind_speed,wind_speed_qual,air_temperature,air_temperature_qual,country
0,2003,703160,25624,20030101,0000,SY-MT,010,5,N,5.2,5,-0.6,5,US
1,2003,703160,25624,20030101,0017,FM-16,020,1,N,4.6,1,-2.0,1,US
2,2003,703160,25624,20030101,0053,FM-15,010,5,N,5.2,5,-2.8,5,US
3,2003,703160,25624,20030101,0100,NSRDB,999,9,9,999.9,9,999.9,9,US
4,2003,703160,25624,20030101,0153,FM-15,010,5,N,6.2,5,-2.2,5,US
5,2003,703160,25624,20030101,0200,NSRDB,999,9,9,999.9,9,999.9,9,US
6,2003,703160,25624,20030101,0253,FM-15,010,5,N,7.2,5,-3.3,5,US
7,2003,703160,25624,20030101,0300,NSRDB,999,9,9,999.9,9,999.9,9,US
8,2003,703160,25624,20030101,0353,FM-15,020,5,N,6.2,5,-1.1,5,US
9,2003,703160,25624,20030101,0400,NSRDB,999,9,9,999.9,9,999.9,9,US


## 2.3 Aggregate Temperature

We now aggregate the min, max and average air temperature per country and year. We will use a simple `WHERE` condition to ignore all records with invalid air temperature. Of course, we remember a more efficient overall solution. In order to make the example more complicated, we chose to ignore the simpler solution.

In [23]:
query = """
    SELECT
        year,
        country,
        min(air_temperature) AS min_air_temperature,
        max(air_temperature) AS max_air_temperature,
        avg(air_temperature) AS avg_air_temperature
    FROM joined_weather
    WHERE air_temperature_qual = '1'
    GROUP BY year,country
"""

# Create a Spark DataFrame for the SQL query above and register it as a temp view called 'year_country_temperature'
year_country_temperature = spark.sql(query)
year_country_temperature.createOrReplaceTempView("year_country_temperature")

# Display first 10 records from the newly created temp view
spark.sql("SELECT * FROM year_country_temperature LIMIT 10").toPandas()

,year,country,min_air_temperature,max_air_temperature,avg_air_temperature
0,2003,NL,-14.3,36.0,10.847775
1,2003,US,-44.0,41.0,6.856936
2,2003,BE,-11.3,36.3,10.945630
3,2003,AU,-17.6,37.4,10.012979
4,2003,EZ,-16.0,37.0,8.918003
5,2003,IC,-9.0,20.0,7.012094
6,2003,LU,-13.0,37.4,10.490859
7,2003,GM,-15.3,36.0,8.984825
8,2003,PO,-2.0,39.0,15.615641
9,2003,UK,-7.2,37.4,11.073555


## 2.4 Aggregate Wind

As the next step, create a similar query for calculating the min, max and average wind speed per country and year.

In [22]:
query = """
    SELECT
        year,
        country,
        min(wind_speed) AS min_wind_speed,
        max(wind_speed) AS max_wind_speed,
        avg(air_temperature) AS avg_wind_speed
    FROM joined_weather
    WHERE wind_speed_qual = '1'
    GROUP BY year,country
"""

# Create a Spark DataFrame for the SQL query above and register it as a temp view called 'year_country_wind'
year_country_wind = spark.sql(query)
year_country_wind.createOrReplaceTempView("year_country_wind")

# Display first 10 records from the newly created temp view
spark.sql("SELECT * FROM year_country_wind LIMIT 10").toPandas()

,year,country,min_wind_speed,max_wind_speed,avg_wind_speed
0,2003,NL,0.0,33.4,40.490046
1,2003,US,0.0,35.0,68.044499
2,2003,BE,0.0,19.0,20.184212
3,2003,AU,0.0,22.6,34.653486
4,2003,EZ,0.0,16.5,16.807488
5,2003,IC,0.0,30.9,14.995883
6,2003,LU,0.0,16.5,27.939163
7,2003,GM,0.0,16.5,16.381248
8,2003,PO,0.0,20.0,24.131470
9,2003,UK,0.0,22.0,11.622068


## 2.5 Join Final Result

So far, we have created the two temp views `year_country_temperature` and `year_country_wind`. Both contain some aggregated attributes per country and year. Now join together both data frames using the columns `country` and `year`. Here we chose an *outer join*, since both sides of the join could provide some relevant information without the other side. The final result should contain the following columns

* `year`
* `country`
* `min_wind_speed`
* `max_wind_speed`
* `avg_wind_speed`
* `min_air_temperature`
* `max_air_temperature`
* `avg_air_temperature`

The function SQL function `COALESCE` can be used to merge the columns `year` and `country` from the left and right sides of the joint into two final columns `year` and `country`.

In [24]:
query = """
    SELECT
        COALESCE(wind.year, temp.year) AS year,
        COALESCE(wind.country, temp.country) AS country,
        wind.min_wind_speed,
        wind.max_wind_speed,
        wind.avg_wind_speed,
        temp.min_air_temperature,
        temp.max_air_temperature,
        temp.avg_air_temperature
    FROM year_country_wind wind
    FULL OUTER JOIN year_country_temperature temp ON wind.year = temp.year AND wind.country = temp.country
"""

# Execute the query
result = spark.sql(query)
# Convert the result to a Pandas dataframe - this time without a 'limit' 
result.toPandas()

,year,country,min_wind_speed,max_wind_speed,avg_wind_speed,min_air_temperature,max_air_temperature,avg_air_temperature
0,2003,AS,0.0,14.4,24.965552,0.3,45.8,23.025439
1,2003,AU,0.0,22.6,34.653486,-17.6,37.4,10.012979
2,2003,BE,0.0,19.0,20.184212,-11.3,36.3,10.945630
3,2003,CA,0.0,27.8,118.253679,-40.1,35.8,1.453475
4,2003,DA,0.0,23.0,15.369517,-16.0,30.0,8.552005
...,...,...,...,...,...,...,...,...
480,2019,SF,0.0,10.8,37.555696,2.7,42.5,20.635556
481,2019,SV,0.0,32.0,-4.371690,-36.2,16.0,-4.632360
482,2019,SW,0.0,20.0,4.055555,-39.3,29.3,2.710664
483,2019,UK,0.0,17.0,11.209918,-7.2,35.0,11.100897


# 3.0 Common Table Expression

The following section will provide an SQL query, which essentially performs the very same calculation. But we implement all steps within a single query using *Common Table Expressions*, CTEs. A CTE can be seen as a local temp view, which is locally scoped to be only accessible within a single query. Therefore, a CTE can be interpreted as some sort of *table valued function* (but without a parameter).

## 3.1 Air Temperature

In order to provide an instructive example, let's start with a simpler query which only calculates the metrics related to the air temperature.

In [29]:
query = """
-- Extract measurements
WITH weather AS (
    SELECT
        year,
        substring(value,5,6) AS usaf,
        substring(value,11,5) AS wban,
        substring(value,16,8) AS `date`,
        substring(value,24,4) AS `time`,
        substring(value,42,5) AS report_type,
        substring(value,61,3) AS wind_direction,
        substring(value,64,1) AS wind_direction_qual,
        substring(value,65,1) AS wind_observation,
        CAST(substring(value,66,4) AS FLOAT) / 10.0 AS wind_speed,
        substring(value,70,1) AS wind_speed_qual,
        CAST(substring(value,88,5) AS FLOAT) / 10.0 AS air_temperature,
        substring(value,93,1) AS air_temperature_qual
    FROM raw_weather
),
-- Join measurements and master data
joined_weather AS (
    SELECT
        weather.*,
        CTRY AS country
    FROM weather
    LEFT JOIN stations ON weather.usaf = stations.usaf AND weather.wban = stations.wban
)
-- Calculate min/max/avg air temperature per county and year
SELECT
    year,
    country,
    min(air_temperature) AS min_air_temperature,
    max(air_temperature) AS max_air_temperature,
    avg(air_temperature) AS avg_air_temperature
FROM joined_weather
WHERE air_temperature_qual = '1'
GROUP BY year,country
"""

# Execute the query and display result
result = spark.sql(query)
result.toPandas()

,year,country,min_air_temperature,max_air_temperature,avg_air_temperature
0,2003,NL,-14.3,36.0,10.847775
1,2003,US,-44.0,41.0,6.856936
2,2003,BE,-11.3,36.3,10.945630
3,2003,AU,-17.6,37.4,10.012979
4,2003,EZ,-16.0,37.0,8.918003
...,...,...,...,...,...
478,2019,CH,13.0,35.0,27.250057
479,2019,AS,2.7,45.8,23.224927
480,2019,SF,2.7,42.5,20.635556
481,2019,RS,-25.9,29.9,4.778747


## 3.2 Full Query

In the next and final step, we construct a more complicated query, which also calculates the wind speed metrics and which performs the final join between the wind speed and air temperature metrics.

In [27]:
query = """
-- Extract measurements
WITH weather AS (
    SELECT
        year,
        substring(value,5,6) AS usaf,
        substring(value,11,5) AS wban,
        substring(value,16,8) AS `date`,
        substring(value,24,4) AS `time`,
        substring(value,42,5) AS report_type,
        substring(value,61,3) AS wind_direction,
        substring(value,64,1) AS wind_direction_qual,
        substring(value,65,1) AS wind_observation,
        CAST(substring(value,66,4) AS FLOAT) / 10.0 AS wind_speed,
        substring(value,70,1) AS wind_speed_qual,
        CAST(substring(value,88,5) AS FLOAT) / 10.0 AS air_temperature,
        substring(value,93,1) AS air_temperature_qual
    FROM raw_weather
),
-- Join measurements and master data
joined_weather AS (
    SELECT
        weather.*,
        CTRY AS country
    FROM weather
    LEFT JOIN stations ON weather.usaf = stations.usaf AND weather.wban = stations.wban
),
-- Aggregate air temperature
year_country_temperature AS (
    SELECT
        year,
        country,
        min(air_temperature) AS min_air_temperature,
        max(air_temperature) AS max_air_temperature,
        avg(air_temperature) AS avg_air_temperature
    FROM joined_weather
    WHERE air_temperature_qual = '1'
    GROUP BY year,country
),
-- Aggregate wind speed
year_country_wind AS (
    SELECT
        year,
        country,
        min(wind_speed) AS min_wind_speed,
        max(wind_speed) AS max_wind_speed,
        avg(air_temperature) AS avg_wind_speed
    FROM joined_weather
    WHERE wind_speed_qual = '1'
    GROUP BY year,country
)
-- Join aggeragted wind speed and aggregated air temperature
SELECT
    COALESCE(wind.year, temp.year) AS year,
    COALESCE(wind.country, temp.country) AS country,
    wind.min_wind_speed,
    wind.max_wind_speed,
    wind.avg_wind_speed,
    temp.min_air_temperature,
    temp.max_air_temperature,
    temp.avg_air_temperature
FROM year_country_wind wind
FULL OUTER JOIN year_country_temperature temp ON wind.year = temp.year AND wind.country = temp.country
"""

# Execute the query and display result
result = spark.sql(query)
result.toPandas()

,year,country,min_wind_speed,max_wind_speed,avg_wind_speed,min_air_temperature,max_air_temperature,avg_air_temperature
0,2003,AS,0.0,14.4,24.965552,0.3,45.8,23.025439
1,2003,AU,0.0,22.6,34.653486,-17.6,37.4,10.012979
2,2003,BE,0.0,19.0,20.184212,-11.3,36.3,10.945630
3,2003,CA,0.0,27.8,118.253679,-40.1,35.8,1.453475
4,2003,DA,0.0,23.0,15.369517,-16.0,30.0,8.552005
...,...,...,...,...,...,...,...,...
480,2019,SF,0.0,10.8,37.555696,2.7,42.5,20.635556
481,2019,SV,0.0,32.0,-4.371690,-36.2,16.0,-4.632360
482,2019,SW,0.0,20.0,4.055555,-39.3,29.3,2.710664
483,2019,UK,0.0,17.0,11.209918,-7.2,35.0,11.100897
